In [33]:
from Bio import Entrez
import urllib.parse
import time
import xmltodict
from xml.dom import minidom

In [34]:
Entrez.email = "yiyuanp1@student.unimelb.edu.au"

In [35]:
# import NIO dictionary token id and canonicals
# the canonicals with be the elements inside the queries to be encoded 
xml_path = "/Users/yidesdo21/Projects/inputs/dictionary/"

with open(xml_path+"nio_ado_case.xml") as f:
    nio_xml = f.read()

nio_parsed = xmltodict.parse(nio_xml)

In [36]:
nio_dict = nio_parsed["synonym"]["token"]

In [37]:
nio_dict[:5]

[OrderedDict([('@id', 'NDDO:Geriatric_Depression_Scale_Assessment_design'),
              ('@canonical', 'Geriatric Depression Scale Assessment design'),
              ('variant',
               OrderedDict([('@base',
                             'Geriatric Depression Scale Assessment design')]))]),
 OrderedDict([('@id', 'NDDO:NDDO_00000021'),
              ('@canonical', 'visit'),
              ('variant', OrderedDict([('@base', 'visit')]))]),
 OrderedDict([('@id', 'NDDO:NDDO_00000022'),
              ('@canonical', 'diagnostic data'),
              ('variant', OrderedDict([('@base', 'diagnostic data')]))]),
 OrderedDict([('@id', 'NDDO:NDDO_00000023'),
              ('@canonical', 'MRI image'),
              ('variant', OrderedDict([('@base', 'MRI image')]))]),
 OrderedDict([('@id', 'NDDO:NDDO_00000024'),
              ('@canonical', 'clinical data'),
              ('variant', OrderedDict([('@base', 'clinical data')]))])]

In [38]:
# for now, we only use the concept ids from AlzheimerOntology
token_ids, canos = list(), list()

for nio in nio_dict:
    token_id, cano = nio["@id"], nio["@canonical"]
    if token_id.startswith("AlzheimerOntology"):
        token_ids.append(token_id)
        canos.append(cano)
#         print(token_id)
#         print(cano)
#         print(nio)
#         print("-------------")

In [39]:
len(token_ids)

1306

In [40]:
len(canos)

1306

In [41]:
len(canos)

1306

In [42]:
# why "alzheimer's disease" is not working? 
# esearch should return 4 uids in the website, but it return 0 here
# queries = ["alzheimer's disease", "vomiting", "apoe", "APoE"]
queries = canos[:305]   # test
# queries = canos
encodings = list()

for query in queries:
    encoding = urllib.parse.quote_plus(query)  # it doesn't matter using "%20" or "+" for the space
    encodings.append(encoding)
    
query_enc = dict(zip(encodings, queries))

In [19]:
print(encodings)

['AKT1', 'Difficulty+in+learning+new+things', 'Alexy', 'AGER', 'PLAT', 'SIRT4', 'NRG1', 'Early+onset+Alzheimer%27s+Disease', 'Taking+longer+than+before+to+complete+normal+daily+tasks', 'IRS1']


In [43]:
len(encodings)

1306

In [22]:
print(query_enc)

{'AKT1': 'AKT1', 'Difficulty+in+learning+new+things': 'Difficulty in learning new things', 'Alexy': 'Alexy', 'AGER': 'AGER', 'PLAT': 'PLAT', 'SIRT4': 'SIRT4', 'NRG1': 'NRG1', 'Early+onset+Alzheimer%27s+Disease': "Early onset Alzheimer's Disease", 'Taking+longer+than+before+to+complete+normal+daily+tasks': 'Taking longer than before to complete normal daily tasks', 'IRS1': 'IRS1'}


In [44]:
len(query_enc)

1304

In [45]:
database = "mesh"

In [48]:
# can we link the uids to the heading uid? how to know which level is what we need?
start_time = time.time()
database = "mesh"
Database = database.upper()+":"
uid_lists = list()
cnt = 0

for e in encodings:
#     print(e)
#     print(query_enc.get(e))
    handle = Entrez.esearch(db=database, term=e)  # handle is a wrapper around text information
    record = Entrez.read(handle)
    handle.close()
    count, idlist = record["Count"], record["IdList"]
    
    # turn the MeSH id into the format in the PTC identifier
    uid_list = [Database+chr(int(i[:2]))+i[2:] for i in idlist if len(i) >= 8]        
    uid_lists.append(uid_list)
    
#     print(count)
#     print(idlist)
#     print(len(uid_list))
#     print(uid_list)
#     print("-----------")
    cnt += 1
    if cnt % 100 == 0:
        print("Processing %s uids" % cnt)

elapsed_time = time.time() - start_time
print("Time of running the codes:", round(elapsed_time,2), "secs.")

Processing 100 uids
Processing 200 uids
Processing 300 uids


KeyboardInterrupt: 

In [49]:
len(uid_lists)

305

In [50]:
print(uid_lists)

[['MESH:D051057', 'MESH:C532657', 'MESH:C516104', 'MESH:C514862', 'MESH:C508191', 'MESH:C494920', 'MESH:C494918', 'MESH:C494917', 'MESH:C484523', 'MESH:C417562', 'MESH:C074500'], [], [], ['MESH:C452619'], ['MESH:C111406', 'MESH:C067262'], ['MESH:C525357', 'MESH:C514813'], ['MESH:D020890', 'MESH:C551996', 'MESH:C490582', 'MESH:C490581', 'MESH:C490580', 'MESH:C117881', 'MESH:C094131'], [], [], ['MESH:C580705', 'MESH:C526296', 'MESH:C526295', 'MESH:C526294', 'MESH:C526292', 'MESH:C072216'], [], [], [], ['MESH:C547408', 'MESH:C510282', 'MESH:C049312'], [], ['MESH:C559639', 'MESH:C542006', 'MESH:C524492', 'MESH:C435197', 'MESH:C121799', 'MESH:C103631', 'MESH:C098681', 'MESH:C070440'], ['MESH:D056566', 'MESH:C525046', 'MESH:C466093'], ['MESH:D060086', 'MESH:D059038', 'MESH:D055008', 'MESH:D053260', 'MESH:D051039', 'MESH:D046936', 'MESH:D046009', 'MESH:D044383', 'MESH:D032407', 'MESH:D032324', 'MESH:D032304', 'MESH:D032107', 'MESH:D031965', 'MESH:D031885', 'MESH:D031881', 'MESH:D031329', 'MES

In [52]:
# link the MeSH UIDs to the NIO dictionary 
## create an XML file that links NIO UIDs with PTC UIDs
# <token id="AlzheimerOntology:Subtypes" canonical="Alzheimer disease">
# 		<ptc id="MESH:D000544"></ptc>
# 		<ptc id="MESH:C567463"></ptc>
# </token>

# create file
xmldoc = minidom.Document()

# creat root element
root_element = xmldoc.createElement('linking_uids')
xmldoc.appendChild(root_element)

uid_cnt = len(token_ids[:205])  # test
# uid_cnt = len(token_ids)  # test

for i in range(uid_cnt):
    token_id = token_ids[i]
    productChild = xmldoc.createElement('nio')
    productChild.setAttribute('id', token_id)  # attribute, value   
    
    productChild.setAttribute('canonical', canos[i])
    root_element.appendChild(productChild)    
    
    for uid in uid_lists[i]:
        product_grandChild = xmldoc.createElement('ptc')

        # insert ptc uid into element
        product_grandChild.setAttribute('id', uid)
        productChild.appendChild(product_grandChild)     
        
xml_str = xmldoc.toprettyxml(indent ="\t")

# save file
save_path_file = "nio_ptc_uids.xml"
dict_path = "/Users/yidesdo21/Projects/inputs/dictionary/"

with open(dict_path+save_path_file, "w") as f:
    f.write(xml_str)        